In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# 이미지 크기와 경로 설정
IMG_SIZE = 128
BOAT_DIR = 'boat_on_sea_images'  # 바다 위의 보트 이미지 경로
OTHER_DIR = 'non_boat_images'  # 보트가 아닌 이미지 경로

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Failed to process image {filename}: {e}")
    return images, labels

# 보트와 보트가 아닌 이미지 불러오기
boat_images, boat_labels = load_images_from_folder(BOAT_DIR, 1)  # 1 for boat
other_images, other_labels = load_images_from_folder(OTHER_DIR, 0)  # 0 for other

# 이미지와 레이블 결합
X = np.array(boat_images + other_images)
y = np.array(boat_labels + other_labels)

# 데이터 정규화 (0~1 사이 값으로 변환)
X = X.astype('float32') / 255.0

# 데이터셋 분할 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 이진 분류
])

# 모델 컴파일
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=32)

# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Test loss: {test_loss:.2f}")

# 학습 과정 시각화

# 정확도 그래프
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 손실도 그래프
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지 크기와 경로 설정
IMG_SIZE = 128
BOAT_DIR = 'boat_on_sea_images'  # 바다 위의 보트 이미지 경로
OTHER_DIR = 'non_boat_images'    # 보트가 아닌 이미지 경로

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Failed to process image {filename}: {e}")
    return images, labels

# 보트와 보트가 아닌 이미지 불러오기
boat_images, boat_labels = load_images_from_folder(BOAT_DIR, 1)  # 1 for boat
other_images, other_labels = load_images_from_folder(OTHER_DIR, 0)  # 0 for other

# 이미지와 레이블 결합
X = np.array(boat_images + other_images)
y = np.array(boat_labels + other_labels)

# 데이터 정규화 (0~1 사이 값으로 변환)
X = X.astype('float32') / 255.0

# 데이터셋 분할 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 이진 분류
])

# 모델 컴파일
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=32)

# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy:.2f}")
print(f"Test loss: {test_loss:.2f}")

# 예측 수행
y_pred = model.predict(X_test)
y_pred_classes = np.round(y_pred).astype(int).flatten()

# 혼돈 행렬 계산
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print(conf_matrix)

# 혼돈 행렬 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', cbar=False, 
            xticklabels=['Non-Boat', 'Boat'], yticklabels=['Non-Boat', 'Boat'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# 분류 보고서 출력
print(classification_report(y_test, y_pred_classes, target_names=['Non-Boat', 'Boat']))
